### Python code style
1. http://docs.python-guide.org/en/latest/writing/style/
2. http://pep8.org/
3. https://www.python.org/dev/peps/pep-0008/

## Классы

Подробнее о классах (и в целом очень хороший курс)

https://compscicenter.ru/courses/python/2015-autumn/classes/1477/

### Простой пример

In [1]:
import numpy as np

In [2]:
class Transformer:
    
    def __init__(self, power=2):
        self.power = power
    
    # self is important
    def transform(self, x):
        return x ** self.power

In [3]:
trans = Transformer()
x = np.arange(5)
trans.transform(x)

array([ 0,  1,  4,  9, 16])

### Атрибуты экземпляра и атрибуты класса

In [4]:
class Transformer:
    num_times_called = 0
    
    def __init__(self, power=2):
        self.power = power
    
    def transform(self, x):
        Transformer.num_times_called += 1
        return x ** self.power

In [5]:
trans1 = Transformer()
trans1.transform(x)

array([ 0,  1,  4,  9, 16])

In [6]:
trans2 = Transformer()
trans2.transform(x)

array([ 0,  1,  4,  9, 16])

In [7]:
trans2.num_times_called

2

### Наследование

In [8]:
class StandartTrasnformer(Transformer):
    def __init__(self, power=2, is_std=False, ):
        self.is_std = is_std
        super(StandartTrasnformer, self).__init__(power)
    
    def transform(self, x):
        # using here -= is bad as 
        # it can induce casting problems
        x = x - x.mean()
        if self.is_std:
            x /= x.std()
        return x ** self.power

In [9]:
trans = StandartTrasnformer(power=1, is_std=False)
trans.transform(x)

array([-2., -1.,  0.,  1.,  2.])

In [10]:
# we wanted other behaviour
trans.num_times_called

2

In [11]:
class StandartTrasnformer(Transformer):
    # https://www.python.org/dev/peps/pep-0008/#string-quotes
    """Some fancy transformer"""
    
    def __init__(self, power=2, is_std=False, ):
        self.is_std = is_std
        super(StandartTrasnformer, self).__init__(power)
    
    def transform(self, x):
        x = x - x.mean()
        if self.is_std:
            x /= x.std()
        return super(StandartTrasnformer, self).transform(x)

In [12]:
trans = StandartTrasnformer(power=1, is_std=False)
trans.transform(x)

array([-2., -1.,  0.,  1.,  2.])

In [13]:
trans.num_times_called

3

### Приватные методы

In [14]:
class A:
    def __init__(self):
        self.public_var = 0
        self._private_var = 1
        self.__very_private_var = 2

In [15]:
print('public:', A().public_var)
print('private:', A()._private_var)
print('very private:', A().__very_private_var)

public: 0
private: 1


AttributeError: 'A' object has no attribute '__very_private_var'

In [16]:
print('very private:', A()._A__very_private_var)

very private: 2


### Property

In [17]:
import os


class Path:
    def __init__(self, current):
        self.current = current
    
    def __repr__(self):
        return 'Path({})'.format(self.current)
    
    @property
    def parent(self):
        return Path(os.path.dirname(self.current))

In [18]:
image_path = Path('Images/picture.png')
image_path

Path(Images/picture.png)

In [19]:
image_path.parent

Path(Images)

### Class dict

In [20]:
print(StandartTrasnformer.__doc__)
print(StandartTrasnformer.__name__)

Some fancy transformer
StandartTrasnformer


In [21]:
StandartTrasnformer.__dict__

mappingproxy({'__doc__': 'Some fancy transformer',
              '__init__': <function __main__.StandartTrasnformer.__init__>,
              '__module__': '__main__',
              'transform': <function __main__.StandartTrasnformer.transform>})

In [22]:
trans.__dict__

{'is_std': False, 'power': 1}

In [23]:
vars(trans)

{'is_std': False, 'power': 1}

In [24]:
vars(StandartTrasnformer)

mappingproxy({'__doc__': 'Some fancy transformer',
              '__init__': <function __main__.StandartTrasnformer.__init__>,
              '__module__': '__main__',
              'transform': <function __main__.StandartTrasnformer.transform>})

### isinstance issubclass

In [25]:
isinstance(trans, Transformer)

True

In [26]:
isinstance(trans, StandartTrasnformer)

True

In [27]:
issubclass(StandartTrasnformer, Transformer)

True

### Множественное наследование и Mixins

Лучше не использовать, но иногда можно

In [28]:
class ParameterRepresetationMixin:
    template = '{:>10} | {:>5}'
    
    def pretty_print(self):
        template = ParameterRepresetationMixin.template
        for param in vars(self):
            print(template.format(param, vars(self)[param]))

In [29]:
class StandartTrasnformer(ParameterRepresetationMixin, Transformer):
    """Some fancy transformer"""
    
    def __init__(self, power=2, is_std=False, ):
        self.is_std = is_std
        super(StandartTrasnformer, self).__init__(power)
    
    def transform(self, x):
        x = x - x.mean()
        if self.is_std:
            x /= x.std()
        return super(StandartTrasnformer, self).transform(x)

In [30]:
trans = StandartTrasnformer(power=1, is_std=False)
trans.transform(x)

array([-2., -1.,  0.,  1.,  2.])

In [31]:
trans.pretty_print()

    is_std |     0
     power |     1


#### Но некоторые используют для другого, например, sklearn

In [32]:
class BaseEstimator(object):
    pass


class TransformerMixin(object):
    """Mixin class for all transformers in scikit-learn."""
    
    def fit_transform(self, X, y=None, **fit_params):
        pass
    

# mixin is the second!
class Imputer(BaseEstimator, TransformerMixin):
    pass


class MinMaxScaler(BaseEstimator, TransformerMixin):
    pass

In [33]:
scaler = MinMaxScaler()
imp = Imputer()

In [34]:
# we can check that these are transformers
print(isinstance(scaler, TransformerMixin))
print(isinstance(imp, TransformerMixin))

True
True
